In [137]:
## from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import os
import io

import os

import sys

import string

import numpy as np

import pandas as pd

from tensorflow import keras

from __future__ import print_function

from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping

from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding


#
# with open(f"{cwd}\inspiration\{dataroot[0]}").read().replace('\n', '. ') as file:
#     print(file)
cwd = os.getcwd()
dataroot = os.listdir(cwd + "\inspiration")

sentData = []

for i in range(0, len(dataroot)):
    print(f"{cwd}\inspiration\{dataroot[i]}")
    data2 = open(f"{cwd}\inspiration\{dataroot[i]}", encoding="utf8").read().replace(',', '').lower().split('\n')
    sentData.append(data2)



C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Andre 3000_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\ASAP Rocky_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\business.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Drake_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\eminemlyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\getIntoIt.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Isaiah Rashad_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\J Cole_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Logic_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Mac Miller_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\melt_session.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Nas_lyrics.txt
C:\Users\sebbe\Desktop\ME

In [138]:
print(sentData[2][:])


["you 'bout to witness hip hop in its most purest", 'most rawest form flow almost flawless', 'most hardest most honest known artist', 'chip off the old block but old doc is', 'back!', 'looks like batman brought his own robin', "oh god saddam's got his own laden", 'with his own private plane his own pilot', 'set to blow college dorm room doors off their hinges', 'oranges peach pears plums syringes', "yeah here i come i'm inches", 'away from you dear fear none', 'hip hop is in a state of nine-one-one so', "let's get down to business", "i don't got no time to play around what is this?", "must be a circus in town let's shut that shit down", 'on these clowns can i getta witness?', '(hell yeah!)', "let's get down to business", "i don't got no time to play around what is this?", "must be a circus in town let's shut that shit down", 'on these clowns can i getta witness?', '(hell yeah!)', 'quick gotta move fast gotta perform miracles', 'gee wilikers dre holy bat syllables', 'look at all the bul

In [139]:
textAsList = []
textAsList.clear()

def extractTextAsList(text):
    global textAsList
    textAsList += [w for w in text.split(' ') if w.strip() !='' or w=='\n' or w=='.']
    
    
        
    


In [141]:

extractTextAsList(sentData[4][0])
print(textAsList)
len(textAsList)

["i'm", "beginnin'", 'to', 'feel', 'like', 'a', 'rap', 'god', 'rap', 'god']


10

In [142]:
for index in range(0, len(sentData)):
    for j in range(0, len(sentData[index])):
        extractTextAsList(sentData[index][j])
        
print("Total amount of words: ", len(textAsList))
print(textAsList[11330])

Total amount of words:  391378
and


In [143]:
frequencies = {}
frequencies.clear()


for word in textAsList:
    frequencies[word] = frequencies.get(word, 0) + 1
    
print(len(frequencies))

print("How many fucks?:", frequencies['fuck']+ frequencies['fucking'] + frequencies['fucker'] + frequencies['fucks'])
print("How many nibbas?: ", frequencies['nigga'] + frequencies['niggers'] + frequencies['niggas'])

24534
How many fucks?: 2119
How many nibbas?:  2713


In [144]:
uncommonWords = set()
minFreq = 7
minSeq = 5
BATCH_SIZE = 32

uncommonWords = set([key for key in frequencies.keys() if frequencies[key] < minFreq])
words = sorted(set([key for key in frequencies.keys() if frequencies[key] >= minFreq]))

numWords = len(words)

wordIndices = dict((w, i) for i, w in enumerate(words))
indicesWord = dict((i, w) for i, w in enumerate(words))

#print(wordIndices)
#print(indicesWord)

print(f'Words with less than {minFreq} appearances: {len(uncommonWords)}')

print(f'Words with more than {minFreq} appearances: {len(words)}')

validSeqs = []
endSeqWords = []

for i in range(len(textAsList) - minSeq):
    endSlice = i + minSeq + 1
    
    if len(set(textAsList[i: endSlice]).intersection(uncommonWords)) == 0:
        validSeqs.append(textAsList[i: i + minSeq])
        endSeqWords.append(textAsList[i + minSeq])
        
print(f'Valid sequences of size {minSeq}: {len(validSeqs)}')

Words with less than 7 appearances: 20588
Words with more than 7 appearances: 3946
Valid sequences of size 5: 233032


In [145]:
xTrain, xTest, yTrain, yTest = train_test_split(validSeqs, endSeqWords, test_size=0.02, random_state=42)

print(xTrain[2:5])

# Data generator for fit and evaluate
def generator(sentenceList, nextWordList, batchSize):
    index = 0
    
    while True:
        x = np.zeros((batchSize, minSeq), dtype=np.int32)
        y = np.zeros((batchSize), dtype=np.int32)
        
        for i in range(batchSize):
            for t, w in enumerate(sentenceList[index % len(sentenceList)]):
                x[i, t] = wordIndices[w]
                
            y[i] = wordIndices[nextWordList[index % len(sentenceList)]]
            
            index = index + 1
            
        yield x, y

# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    #helper function to sample and index from a probabillity array
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    
    expPreds = np.exp(preds)
    
    preds = exp_preds / np.sum(expPreds)
    
    probabIndx = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probabIndx)

def onEpochEnd(epoch, logs):
    
    #Function invoked at end of each epoch. Prints generated text.
    fxFile.write('\n----- Generating text after Epoch: %d\n' % epoch)
    
    #Pick seed at random index
    seedIndex = np.random.randint(len(xTrain+xTest))
    seed = (xTrain+xTest)[seedIndex]
    
    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        
        fxFile.write('----- Diversity: ' + str(diversity) + '\n')
        fxFile.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        fxFile.write(' '.join(sentence))
        
        for i in range(50):
            xPred = np.zeros((1, minSeq))
            
            for t, word in enumerate(sentence):
                
                xPred[0, t] = wordIndices[word]
                
            preds = model.predict(xPred, verbose=0)[0]
            
            nextIndex = sample(preds, diversity)
            
            nextWord = indicesWord[nextIndex]
            
            sentence = sentence[1:]
            
            sentence.append(nextWord)
            
            fxFile.write('\n')
        
        fxFile.write('='*80 + '\n')
        fxFile.flush()
    
            

[['with', 'a', 'truck', 'ha', 'get'], ['more', 'i', "can't", 'even', 'feel'], ['stacked', 'my', 'shit', 'list', 'to']]


In [146]:
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding

def get_model():
    print('Build model...')
    
    model = Sequential()
    
    model.add(Embedding(input_dim=len(words), output_dim=1024))
    
    model.add(Bidirectional(LSTM(128)))
    
    model.add(Dense(len(words)))
    
    model.add(Activation('softmax'))
    
    return model
    
    

In [147]:
model = get_model()

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
"loss{loss:.4f}-acc{accuracy:.4f}-val_loss{val_loss:.4f}-val_acc{val_accuracy:.4f}" % \
(len(words), minSeq, minFreq)

Build model...


In [148]:
checkpoint = keras.callbacks.ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)

print_callback = keras.callbacks.LambdaCallback(on_epoch_end=onEpochEnd)

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)

callbackList = [checkpoint, print_callback, earlyStopping]

fxFile = open('examples.txt', 'w')

model.fit(generator(xTrain, yTest, BATCH_SIZE),
         steps_per_epoch=int(len(validSeqs)/BATCH_SIZE)+1,
         epochs=20,
         callbacks=callbackList,
         validation_data=generator(xTest, yTrain, BATCH_SIZE),
         validation_steps=int(len(yTrain)/BATCH_SIZE)+1)

Epoch 1/20
 145/7283 [..............................] - ETA: 14:42 - loss: 6.9937 - accuracy: 0.0315

UnknownError: Graph execution error:

IndexError: list index out of range
Traceback (most recent call last):

  File "C:\Users\sebbe\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "C:\Users\sebbe\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\sebbe\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1004, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\sebbe\anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 830, in wrapped_generator
    for data in generator_fn():

  File "<ipython-input-145-9ec5815e5f8d>", line 17, in generator
    y[i] = wordIndices[nextWordList[index % len(sentenceList)]]

IndexError: list index out of range


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_31468]